In [1]:
#https://www.kaggle.com/chandrarajsingh/visualization-analysis-insights-predictions

Problem Statement: Predict the survival of a horse based on various observed medical conditions. Load the data  from “horses.csv” and observe whether it contains missing values. The dataset contains many categorical features;  replace them with label encoding. Replace the missing values by the most frequent value in each column. Fit a  decision tree classifier and random forest classifier, and observe the accuracy.

Objective: Learn to fit a decision tree, and compare its accuracy with random forest classifier.

Access: Click on the Labs tab on the left side panel of the LMS. Copy or note the username and password that are  generated. Click on the Launch Lab button. On the page that appears, enter the username and password in the  respective fields, and click Login.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

In [3]:
import os
for dirname, _, filenames in os.walk('I:/PGAIML/Class/Machine Learning/20210508'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

I:/PGAIML/Class/Machine Learning/20210508\.ipynb_checkpoints\Predict the survival of a horse-checkpoint.ipynb
I:/PGAIML/Class/Machine Learning/20210508\Classify Kinematic Data\Classify Kinematic Data.ipynb
I:/PGAIML/Class/Machine Learning/20210508\Classify Kinematic Data\run_or_walk.csv
I:/PGAIML/Class/Machine Learning/20210508\Classify Kinematic Data\.ipynb_checkpoints\Classify Kinematic Data-checkpoint.ipynb
I:/PGAIML/Class/Machine Learning/20210508\College Classification\College Classification.ipynb
I:/PGAIML/Class/Machine Learning/20210508\College Classification\College.csv
I:/PGAIML/Class/Machine Learning/20210508\College Classification\.ipynb_checkpoints\College Classification-checkpoint.ipynb
I:/PGAIML/Class/Machine Learning/20210508\Horse\DecisionTree_Tutor.ipynb
I:/PGAIML/Class/Machine Learning/20210508\Horse\horse.csv
I:/PGAIML/Class/Machine Learning/20210508\Horse\Predict the survival of a horse.ipynb
I:/PGAIML/Class/Machine Learning/20210508\Horse\.ipynb_checkpoints\Predict

In [4]:
df_horse = pd.read_csv("I:/PGAIML/Class/Machine Learning/20210508/Horse/horse.csv")
df_horse.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [5]:
df_horse.isnull().sum() 

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [6]:
#% Of Null
null = df_horse.isnull().sum() 
null = null / len(df_horse)*100
null = null[null>0]
null.sort_values(inplace=True)
null.to_frame()

,0
pulse,8.026756
packed_cell_volume,9.698997
capillary_refill_time,10.702341
total_protein,11.036789
peristalsis,14.715719
mucous_membrane,15.719064
pain,18.394649
abdominal_distention,18.729097
temp_of_extremities,18.729097
respiratory_rate,19.397993


In [7]:
df_horse.dtypes.value_counts()

object     17
float64     7
int64       4
dtype: int64

In [8]:
#unique values of objects
for col in df_horse:
    if (df_horse[col].dtypes == "object"):
        print(col, " : " ,df_horse[col].unique())

surgery  :  ['no' 'yes']
age  :  ['adult' 'young']
temp_of_extremities  :  ['cool' nan 'normal' 'cold' 'warm']
peripheral_pulse  :  ['reduced' nan 'normal' 'absent' 'increased']
mucous_membrane  :  [nan 'pale_cyanotic' 'pale_pink' 'dark_cyanotic' 'normal_pink'
 'bright_red' 'bright_pink']
capillary_refill_time  :  ['more_3_sec' 'less_3_sec' nan '3']
pain  :  ['extreme_pain' 'mild_pain' 'depressed' nan 'severe_pain' 'alert']
peristalsis  :  ['absent' 'hypomotile' nan 'hypermotile' 'normal']
abdominal_distention  :  ['severe' 'slight' 'none' nan 'moderate']
nasogastric_tube  :  [nan 'none' 'slight' 'significant']
nasogastric_reflux  :  [nan 'less_1_liter' 'none' 'more_1_liter']
rectal_exam_feces  :  ['decreased' 'absent' 'normal' nan 'increased']
abdomen  :  ['distend_large' 'other' 'normal' nan 'firm' 'distend_small']
abdomo_appearance  :  [nan 'cloudy' 'serosanguious' 'clear']
outcome  :  ['died' 'euthanized' 'lived']
surgical_lesion  :  ['no' 'yes']
cp_data  :  ['no' 'yes']


In [9]:
df_horse['pain'].value_counts()

mild_pain       67
depressed       59
extreme_pain    42
severe_pain     38
alert           38
Name: pain, dtype: int64

In [10]:
#Fill NA columns with Most Frequent
#def fillna(col):
#    col.fillna(col.value_counts().index[0], inplace=True)
#    return col
#df_horse=df_horse.apply(lambda col:fillna(col))

In [11]:
#Fill NA columns with Most Frequent Imputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_horse = pd.DataFrame(imputer.fit_transform(df_horse),columns=df_horse.columns,index=df_horse.index)
df_horse.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,normal_pink,more_3_sec,...,45.0,8.4,cloudy,2.0,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,cool,normal,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,cloudy,2.0,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,cool,normal,dark_cyanotic,more_3_sec,...,74.0,7.4,cloudy,2.0,died,no,4300,0,0,no


In [12]:
df_horse['pain'].value_counts()

mild_pain       122
depressed        59
extreme_pain     42
severe_pain      38
alert            38
Name: pain, dtype: int64

In [13]:
#% Of Null
null = df_horse.isnull().sum() 
null = null / len(df_horse)*100
#null = null[null>0]
null.sort_values(inplace=True)
null.to_frame()

,0
surgery,0.0
lesion_2,0.0
lesion_1,0.0
surgical_lesion,0.0
outcome,0.0
abdomo_protein,0.0
abdomo_appearance,0.0
total_protein,0.0
packed_cell_volume,0.0
abdomen,0.0


In [14]:
from sklearn.preprocessing import LabelEncoder
col = df_horse.columns
for i in col:
    lb = LabelEncoder()
    lb.fit(df_horse[i].values)
    df_horse[i] = lb.transform(df_horse[i].values)

df_horse

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,0,0,125,23,17,16,1,3,3,2,...,21,36,1,6,0,0,55,0,0,0
1,1,0,214,30,28,9,1,2,4,1,...,26,77,1,6,1,0,13,0,0,0
2,0,0,140,21,3,13,2,2,5,1,...,8,19,1,6,2,0,0,0,0,1
3,1,1,264,29,50,36,0,2,2,2,...,24,24,2,29,0,1,13,0,0,1
4,0,0,133,11,34,20,1,2,2,2,...,48,26,1,6,0,0,36,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,1,0,174,18,39,33,0,2,4,2,...,31,62,1,6,1,0,25,0,0,0
295,0,0,15,10,20,13,1,1,4,2,...,20,17,2,16,1,1,13,0,0,1
296,1,0,84,13,20,17,0,3,4,1,...,36,20,1,6,0,1,25,0,0,0
297,1,0,155,4,33,13,1,3,5,1,...,26,12,2,17,2,1,13,0,0,1


In [15]:
X = df_horse.drop('outcome', axis=1).values
y = df_horse['outcome'].values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

In [17]:
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier

algo = {'LR':LogisticRegression(max_iter=4000),
        'DT':DecisionTreeClassifier(),
        'RFC':RandomForestClassifier(n_estimators=100),
        'SVM':SVC(gamma=0.001),
        'KNN':KNeighborsClassifier(n_neighbors=10)}

In [18]:
for k, v in algo.items():
    model = v
    model.fit(X_train, y_train)
    print('Acurracy of ' + k + ' is {0:.2f}'.format(model.score(X_test, y_test)*100))

Acurracy of LR is 73.33
Acurracy of DT is 71.67
Acurracy of RFC is 76.67
Acurracy of SVM is 76.67
Acurracy of KNN is 75.00


In [ ]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_predict,y_test)

In [19]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_test, y_test)
y_pred = model.predict(X_test)

In [20]:
for x in range(0, 4):
    print("Prediction value " + str(model.predict([X_test[x]])))
    print("Real value " + str(y_test[x]) + "\n")

Prediction value [2]
Real value 2

Prediction value [2]
Real value 2

Prediction value [2]
Real value 2

Prediction value [0]
Real value 0

